# DASH on a large instance

Execution for [2025_04](https://rubinobs.atlassian.net/browse/DM-48556)

This notebook uses the butler only to fetch the tracts/patches, and to fetch the URIs of backing parquet files. Those files are read into the hats-import pipeline directly.

This is done because many `butler.get` results are too large to fit in the memory of a medium or large RSP notebook instance.

Beyond the butler issues, there were additional problems with running the importer on a smaller instance. While these can largely be avoided by running on the dev machines that are available outside notebooks, I think it's a good lesson for how the Rubin data is structured and how we can more efficiently import with our existing tools.

Useful material:
- LINCC notebooks: https://github.com/lsst-sitcom/linccf
- https://github.com/LSSTScienceCollaborations/StackClub/tree/master

In [1]:
from pathlib import Path
# HATS/LSDB
import lsdb
import hats_import.pipeline as runner
from hats_import.catalog.arguments import ImportArguments

from tqdm import tqdm
import pandas as pd

In [2]:
# %pip install -q lsdb hats-import

In [4]:
raw_dir = base_output_dir / "raw"
hats_dir = base_output_dir /  "hats"

raw_dir.mkdir(parents=True, exist_ok=True)
hats_dir.mkdir(parents=True, exist_ok=True)

## nest sources in object catalogs

In [ ]:
diaObject_cat = lsdb.read_hats(os.path.join(hats_dir, "diaObject"))
diaSource_cat = lsdb.read_hats(os.path.join(hats_dir, "diaSource"))
diaForcedSource_cat = lsdb.read_hats(os.path.join(hats_dir, "diaForcedSource"))

In [ ]:
diaObject_cat_nested = diaObject_cat.join_nested(
    diaSource_cat, left_on="diaObjectId", right_on="diaObjectId", nested_column_name="diaSource").join_nested(
    diaForcedSource_cat, left_on="diaObjectId", right_on="diaObjectId", nested_column_name="diaForcedSource")
diaObject_cat_nested

In [ ]:
# diaObject_cat_nested.to_hats(os.path.join(hats_dir, "diaObject_lc"))

In [ ]:
object_cat = lsdb.read_hats(os.path.join(hats_dir, "object"))
source_cat = lsdb.read_hats(os.path.join(hats_dir, "source"))
forcedSource_cat = lsdb.read_hats(os.path.join(hats_dir, "forcedSource"))

In [ ]:
object_cat_nested = object_cat.join_nested(
    #source_cat, left_on="objectId", right_on="objectId", nested_column_name="source").join_nested(
    forcedSource_cat, left_on="objectId", right_on="objectId", nested_column_name="forcedSource")
object_cat_nested

In [ ]:
# object_cat_nested.to_hats(os.path.join(hats_dir, "object_lc"))

In [8]:
client.close()
tmp_path.cleanup()